<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <h3> Objective</h3> <br>
    <b style = "color:#000000"><i>Renew expiring AWS Certificate Manager(ACM) issued SSL Certificates</i></b>
</div>
</center>
<br>

<center><h2><u>Renew SSL Certificate</u></h2></center>

# Steps Overview
1)[ List expiring ACM certificates](#1)<br>
2)[ Renew expiring ACM certificates](#2)

<h3 id="List-all-AWS-Regions"><a id="1" target="_self" rel="nofollow"></a>List all AWS Regions</h3>
<p>This action fetches all AWS Regions to execute Step 1👇. This action will only execute if no <code>region</code> is provided.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput:&nbsp;<code>region</code></p>
</blockquote>

In [3]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')
task.configure(outputName="region")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3 id="List-expiring-ACM-certificates"><a id="1" target="_self" rel="nofollow"></a>List expiring ACM certificates</h3>
<p>Using unSkript's List expiring ACM certificates action, we will fetch all the expiring certificates given a specific number of threshold days.</p>
<blockquote>
<p>This action takes the following parameters: <code>threshold_days</code>, <code>region(Optional)</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>expiring_certificates</code></p>
</blockquote>

In [4]:
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import dateutil
from pydantic import BaseModel, Field
from typing import Dict,List, Optional,Tuple
from unskript.legos.utils import CheckOutput, CheckOutputStatus
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint
import json
import datetime

from beartype import beartype
@beartype
def aws_list_expiring_acm_certificates_printer(output):
    if output is None:
        return
    if isinstance(output, CheckOutput):
        print(output.json())
    else:
        pprint.pprint(output)

@beartype
def aws_list_expiring_acm_certificates(handle, threshold_days: int, region: str=None)-> CheckOutput:
    """aws_list_expiring_acm_certificates returns all the ACM issued certificates which are about to expire given a threshold number of days

        :type handle: object
        :param handle: Object returned from Task Validate

        :type threshold_days: int
        :param threshold_days: Threshold number of days to check for expiry. Eg: 30 -lists all certificates which are expiring within 30 days

        :type region: str
        :param region: Region name of the AWS account

        :rtype: Object containing status, expiring certificates, and error
    """
    arn_list=[]
    domain_list = []
    expiring_certificates_list= []
    expiring_certificates_dict={}
    result_list=[]
    all_regions = [region]
    if region is None or len(region)==0:
        all_regions = aws_list_all_regions(handle=handle)
    for r in all_regions:
        iamClient = handle.client('acm', region_name=r)
        try:
            expiring_certificates_dict={}
            certificates_list = iamClient.list_certificates(CertificateStatuses=['ISSUED'])
            for each_arn in certificates_list['CertificateSummaryList']:
                arn_list.append(each_arn['CertificateArn'])
                domain_list.append(each_arn['DomainName'])
            for cert_arn in arn_list:
                details = iamClient.describe_certificate(CertificateArn=cert_arn)
                for key,value in details['Certificate'].items():
                    if key == "NotAfter":
                        expiry_date = value
                        right_now = datetime.datetime.now(dateutil.tz.tzlocal())
                        diff = expiry_date-right_now
                        days_remaining = diff.days
                        if days_remaining < threshold_days and days_remaining > 0:
                            expiring_certificates_list.append(cert_arn)
            expiring_certificates_dict["region"]= r
            expiring_certificates_dict["certificate"]= expiring_certificates_list
            if len(expiring_certificates_list)!=0:
                result_list.append(expiring_certificates_dict)
        except Exception as e:
            pass
    if len(result_list)!=0:
        return CheckOutput(status=CheckOutputStatus.FAILED,
                   objects=result_list,
                   error=str(""))
    else:
        return CheckOutput(status=CheckOutputStatus.SUCCESS,
                   objects=result_list,
                   error=str(""))


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter_item",
    "threshold_days": "int(threshold_days)"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(outputName="expiring_certificates")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_expiring_acm_certificates, lego_printer=aws_list_expiring_acm_certificates_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Expiring-Certificates"><a id="1" target="_self" rel="nofollow"></a>Create List of Expiring Certificates</h3>
<p>This action filters regions that have no certificates and creates a list of certificates that have to be renewed</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_expiring_certificates</code></p>
</blockquote>

In [6]:
all_expiring_certificates = []
dummy = []
for k,v in expiring_certificates.items():
    if v.status==CheckOutputStatus.FAILED:
        if len(v.objects)!=0:
            dummy = v.objects
            for x in dummy:
                all_expiring_certificates.append(x)
print(all_expiring_certificates)
task.configure(outputName="all_expiring_certificates")

<h3 id="Renew-expiring-ACM-certificates"><a id="2" target="_self" rel="nofollow"></a>Renew expiring ACM certificates</h3>
<p>This action renews <strong>eligible</strong> SSL certificates that are available on ACM. Only exported private certificates can be renewed with this operation. In order to renew your AWS Private CA certificates with ACM, you must first grant the <a href="https://docs.aws.amazon.com/privateca/latest/userguide/PcaWelcome.html" target="_blank" rel="noopener">ACM service principal permission</a> to do so.<br><br><em><strong>A certificate is eligible for automatic renewal subject to the following considerations:</strong></em></p>
<p>1)<span style="color: green;"> ELIGIBLE</span> if associated with another AWS service, such as Elastic Load Balancing or CloudFront.<br>2)<span style="color: green;"> ELIGIBLE</span>if exported since being issued or last renewed.<br>3)<span style="color: green;"> ELIGIBLE</span> if it is a private certificate issued by calling the ACM RequestCertificate API and then exported or associated with another AWS service.<br>4)<span style="color: green;"> ELIGIBLE</span> if it is a private certificate issued through the management console and then exported or associated with another AWS service.<br>5)<span style="color: red;"> NOT ELIGIBLE</span> if it is a private certificate issued by calling the AWS Private CA IssueCertificate API.<br>6)<span style="color: red;"> NOT ELIGIBLE</span> if imported or already expired</p>
<blockquote>
<p>This action takes the following parameters: <code>aws_certificate_arn</code>, <code>region</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>None</code></p>
</blockquote>

In [ ]:
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import dateutil
from pydantic import BaseModel, Field
from typing import Dict,List
import pprint
import datetime

from beartype import beartype
@beartype
def aws_renew_expiring_acm_certificates_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_renew_expiring_acm_certificates(handle, aws_certificate_arn: List, region: str='') -> Dict:
    """aws_renew_expiring_acm_certificates returns all the ACM issued certificates which are about to expire given a threshold number of days

        :type handle: object
        :param handle: Object returned from Task Validate

        :type aws_certificate_arn: List
        :param aws_certificate_arn: ARN of the Certificate

        :type region: str
        :param region: Region name of the AWS account

        :rtype: Result Dictionary of result
    """
    result = {}
    try:
        acmClient = handle.client('acm', region_name=region)
        for arn in aws_certificate_arn:
            acmClient.renew_certificate(CertificateArn=arn)
            result[arn] = "Successfully renewed"
    except Exception as e:
        result["error"] = e
    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "aws_certificate_arn": "iter.get(\\"certificate\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_expiring_certificates",
    "iter_parameter": ["region","aws_certificate_arn"]
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_renew_expiring_acm_certificates, lego_printer=aws_renew_expiring_acm_certificates_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions to list all expiring ACM SSL Certificates and subsequently renewed them. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>